<a href="https://colab.research.google.com/github/ParkSomin23/Voice_EmotionDetection/blob/master/EMO_3_Net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable

import numpy as np

import os
import sys

import pandas as pd
import pickle

import time


In [2]:
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from keras.utils import np_utils, to_categorical

In [3]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [4]:
classes = ['female_angry', 'female_disgust', 'female_fear', 'female_happy',
       'female_neutral', 'female_sad', 'female_surprise', 'male_angry',
       'male_disgust', 'male_fear', 'male_happy', 'male_neutral',
       'male_sad', 'male_surprise']
classes

['female_angry',
 'female_disgust',
 'female_fear',
 'female_happy',
 'female_neutral',
 'female_sad',
 'female_surprise',
 'male_angry',
 'male_disgust',
 'male_fear',
 'male_happy',
 'male_neutral',
 'male_sad',
 'male_surprise']

# Network

In [62]:
class Emo2DNet(nn.Module):
    def __init__(self, n):
        super().__init__()
        self.n = n
        self.conv1 = nn.Conv2d(1, 32, kernel_size = (3, 9), padding=(1,4))
        self.b_norm1 = nn.BatchNorm2d(32)
        
        self.conv2 = nn.Conv2d(32, 32, kernel_size = (3, 9), padding=(1,4))
        self.b_norm2 = nn.BatchNorm2d(32)
        
        self.conv3 = nn.Conv2d(32, 32, kernel_size = (3, 9), padding=(1,4))
        self.b_norm3 = nn.BatchNorm2d(32)
        
        self.conv4 = nn.Conv2d(32, 32, kernel_size = (3, 9), padding=(1,4))
        self.b_norm4 = nn.BatchNorm2d(32)
        
        #self.b_norm5 = nn.BatchNorm2d(32) #
        
        self.fc1 = nn.Linear(32*13, 14)
        #self.fc2 = nn.Linear(64, 32)
        #self.fc3 = nn.Linear(32, 14)
        
    def forward(self, x):
        x = self.conv1(x)
        x = nn.ReLU()(self.b_norm1(x))
        x = nn.MaxPool2d(2)(x)
        x = nn.Dropout2d(p=0.2)(x)

        x = self.conv2(x)
        x = nn.ReLU()(self.b_norm2(x))
        x = nn.MaxPool2d(2)(x)
        x = nn.Dropout2d(p=0.2)(x)
        
        x = self.conv3(x)
        x = nn.ReLU()(self.b_norm1(x))
        x = nn.MaxPool2d(2)(x)
        x = nn.Dropout2d(p=0.2)(x)
        
        x = self.conv4(x)
        x = nn.ReLU()(self.b_norm1(x))
        x = nn.MaxPool2d(2)(x)
        x = nn.Dropout2d(p=0.2)(x)
        
        x = x.view(-1, 32*13)
        #x = nn.ReLU()(self.b_norm5(x))#
        #x = nn.MaxPool1D(2)(x)#
        x = self.fc1(x)
        x = nn.Dropout(p=0.2)(x)
        
        #x = self.fc2(x)
        #x = self.fc3(x)
        
        x = F.log_softmax(x, dim=1)

        return x

In [31]:
class_correct = list(0. for i in range(14))
class_total = list(0. for i in range(14))

def train_model(model, criterion, optimizer, scheduler, num_epochs=5):
    since = time.time()

    best_model_wts = model.state_dict()
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # 각 Epoch은 학습 단계와 검증 단계를 거침
        for phase in ['train', 'valid']:
            if phase == 'train':
                scheduler.step()
                model.train(True)  # 학습 모드 설정
            else:
                model.train(False)  # 검증 모드 설정

            running_loss = 0.0
            running_corrects = 0

            # 데이터 반
            for data in dataloaders[phase]:
                # 입력 데이터 가져오기
                inputs, labels = data

                # 데이터를 Vaariable로 만듦
                if torch.cuda.is_available():
                    inputs = Variable(inputs.cuda())
                    labels = Variable(labels.cuda())
                else:
                    inputs, labels = Variable(inputs), Variable(labels)

                # 파라미터 기울기 초기화
                optimizer.zero_grad()
                
                # forward
                outputs = model(inputs.float())
                _, preds = torch.max(outputs.data, 1)
                loss = criterion(outputs, labels)

                if phase == 'valid':
                  c = (preds == labels.data).squeeze(0)
                  c = c.to('cpu')
                  c = np.array(c).astype('int')

                  for i in range(5): #6....? ㅠㅠ
                    label = labels[i]
                    # print(i, label)
                    class_correct[label] = class_correct[label] + c[i]
                    class_total[label] = class_total[label] + 1


                # 학습 단계에서만 수행, 역전파 + 옵티마이즈(최적화)
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

                # 통계
                running_loss += loss.data
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss.item() / dataset_sizes[phase]
            epoch_acc = running_corrects.item() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # 모델 복사(Deep Copy)
            if phase == 'valid' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    for i in range(14):
      print('Accuracy of %5s : %2d %%' % (classes[i], 100 * class_correct[i] / class_total[i]))

    # 최적의 모델 가중치 로딩
    model.load_state_dict(best_model_wts)
    return model

# Print  confusion matriv heat map plot

In [7]:
def print_confusion_matrix(confusion_matrix, class_names, figsize = (10,7), fontsize=14):
    """Prints a confusion matrix, as returned by sklearn.metrics.confusion_matrix, as a heatmap.
    
    Arguments
    ---------
    confusion_matrix: numpy.ndarray
        The numpy.ndarray object returned from a call to sklearn.metrics.confusion_matrix. 
        Similarly constructed ndarrays can also be used.
    class_names: list
        An ordered list of class names, in the order they index the given confusion matrix.
    figsize: tuple
        A 2-long tuple, the first value determining the horizontal size of the ouputted figure,
        the second determining the vertical size. Defaults to (10,7).
    fontsize: int
        Font size for axes labels. Defaults to 14.
        
    Returns
    -------
    matplotlib.figure.Figure
        The resulting confusion matrix figure
    """
    df_cm = pd.DataFrame(
        confusion_matrix, index=class_names, columns=class_names, 
    )
    fig = plt.figure(figsize=figsize)
    try:
        heatmap = sns.heatmap(df_cm, annot=True, fmt="d")
    except ValueError:
        raise ValueError("Confusion matrix values must be integers.")
        
    heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right', fontsize=fontsize)
    heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right', fontsize=fontsize)
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Gender recode function
def gender(row):
    if row == 'female_disgust' or 'female_fear' or 'female_happy' or 'female_sad' or 'female_surprise' or 'female_neutral':
        return 'female'
    elif row == 'male_angry' or 'male_fear' or 'male_happy' or 'male_sad' or 'male_surprise' or 'male_neutral' or 'male_disgust':
        return 'male'

# Data

# Dataset & DataLoader

In [8]:
class myDataset(Dataset):
    def __init__(self, d, l):
      self.d = d
      self.l = l

    def __getitem__(self, idx):
      data = self.d[idx]
      data = np.transpose(data,(2,0,1))
      l = self.l[idx].argmax()
      #label = emo_dict[l]
      #label = self.l[idx]
      return data, l

    def __len__(self):
      return len(self.d)

#MFCC

1. Best val Acc: 0.453573
2. Best val Acc: 0.103555

ADAM 
1. Best val Acc: 0.46185

In [38]:
import pickle
import pandas as pd

with open("/gdrive/My Drive/mfcc_crema.pkl", 'rb') as f:
    ref_c = pickle.load(f)

with open("/gdrive/My Drive/mfcc_3.pkl", 'rb') as f:
    ref_3 = pickle.load(f)

print(ref_c.shape)
print(ref_3.shape)

ref = pd.read_csv("/gdrive/My Drive/Data_path.csv")
ref = ref.drop(11586)

(7062, 30, 216, 1)
(4524, 30, 216, 1)


In [39]:
mfcc = np.concatenate((ref_3, ref_c), axis=0)
mfcc.shape

(11586, 30, 216, 1)

**1**

In [63]:
X_train, X_test, y_train, y_test = train_test_split(mfcc, ref.labels, test_size=0.25, shuffle=True, random_state=42)


# one hot encode the target 
lb = LabelEncoder()
y_train = np_utils.to_categorical(lb.fit_transform(y_train))
y_test = np_utils.to_categorical(lb.fit_transform(y_test))

# Normalization as per the standard NN process
mean = np.mean(X_train, axis=0)
std = np.std(X_train, axis=0)

X_train = (X_train - mean)/std
X_test = (X_test - mean)/std

**2**

In [71]:
X_train, X_test, y_train, y_test = train_test_split(mfcc, ref.labels, test_size=0.25, shuffle=True, random_state=42)


# one hot encode the target 
lb = LabelEncoder()
y_train = np_utils.to_categorical(lb.fit_transform(y_train))
y_test = np_utils.to_categorical(lb.fit_transform(y_test))

#other normalization
max_data = np.max(X_train)
min_data = np.min(X_train)
X_train = (X_train-min_data)/(max_data-min_data+1e-6)
X_train =  X_train-0.5

max_data = np.max(X_test)
min_data = np.min(X_test)
X_test = (X_test-min_data)/(max_data-min_data+1e-6)
X_test =  X_test-0.5

In [64]:
model_ft = Emo2DNet(30)

if torch.cuda.is_available():
    model_ft = model_ft.cuda()

In [65]:
learning_rate = 0.01
criterion = nn.CrossEntropyLoss()
#optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
optimizer_ft = optim.Adam(model_ft.parameters())
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [66]:
train_dataset = myDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size= 6, shuffle=True)

valid_dataset = myDataset(X_test, y_test)
valid_loader = DataLoader(valid_dataset, batch_size= 6, shuffle=True)

In [67]:
dataset_sizes = {'train':len(train_loader.dataset),'valid':len(valid_loader.dataset)}
dataloaders = {'train':train_loader,'valid':valid_loader}

In [68]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=24)

Epoch 0/23
----------


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:123: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


train Loss: 0.3711 Acc: 0.2402
valid Loss: 0.3894 Acc: 0.2271

Epoch 1/23
----------
train Loss: 0.3161 Acc: 0.3572
valid Loss: 0.3992 Acc: 0.2223

Epoch 2/23
----------
train Loss: 0.2946 Acc: 0.3972
valid Loss: 0.3735 Acc: 0.2475

Epoch 3/23
----------
train Loss: 0.2800 Acc: 0.4223
valid Loss: 0.3682 Acc: 0.2620

Epoch 4/23
----------
train Loss: 0.2705 Acc: 0.4340
valid Loss: 0.3668 Acc: 0.2741

Epoch 5/23
----------
train Loss: 0.2610 Acc: 0.4578
valid Loss: 0.3455 Acc: 0.3141

Epoch 6/23
----------
train Loss: 0.2477 Acc: 0.4833
valid Loss: 0.3379 Acc: 0.3376

Epoch 7/23
----------
train Loss: 0.2377 Acc: 0.5006
valid Loss: 0.3289 Acc: 0.3562

Epoch 8/23
----------
train Loss: 0.2385 Acc: 0.5011
valid Loss: 0.3433 Acc: 0.3155

Epoch 9/23
----------
train Loss: 0.2351 Acc: 0.5074
valid Loss: 0.3420 Acc: 0.3203

Epoch 10/23
----------
train Loss: 0.2349 Acc: 0.4999
valid Loss: 0.3363 Acc: 0.3334

Epoch 11/23
----------
train Loss: 0.2328 Acc: 0.5132
valid Loss: 0.3355 Acc: 0.3338



#Mel Spectrogram

1. Best val Acc: 0.396617

2. Best val Acc: 0.136003

In [48]:
import pickle
import pandas as pd

with open("/gdrive/My Drive/mel_crema.pkl", 'rb') as f:
    mel_c = pickle.load(f)

with open("/gdrive/My Drive/mel_3.pkl", 'rb') as f:
    mel_3 = pickle.load(f)

print(mel_c.shape)
print(mel_3.shape)

ref = pd.read_csv("/gdrive/My Drive/Data_path.csv")
ref = ref.drop(11586)

(7062, 30, 216, 1)
(4524, 30, 216, 1)


In [49]:
mel = np.concatenate((mel_3, mel_c), axis=0)
mel.shape

(11586, 30, 216, 1)

In [50]:
X_train, X_test, y_train, y_test = train_test_split(mel, ref.labels, test_size=0.25, shuffle=True, random_state=42)


# one hot encode the target 
lb = LabelEncoder()
y_train = np_utils.to_categorical(lb.fit_transform(y_train))
y_test = np_utils.to_categorical(lb.fit_transform(y_test))

# Normalization as per the standard NN process
mean = np.mean(X_train, axis=0)
std = np.std(X_train, axis=0)

X_train = (X_train - mean)/std
X_test = (X_test - mean)/std

In [24]:
X_train, X_test, y_train, y_test = train_test_split(mel, ref.labels, test_size=0.25, shuffle=True, random_state=42)


# one hot encode the target 
lb = LabelEncoder()
y_train = np_utils.to_categorical(lb.fit_transform(y_train))
y_test = np_utils.to_categorical(lb.fit_transform(y_test))

#other normalization
max_data = np.max(X_train)
min_data = np.min(X_train)
X_train = (X_train-min_data)/(max_data-min_data+1e-6)
X_train =  X_train-0.5

max_data = np.max(X_test)
min_data = np.min(X_test)
X_test = (X_test-min_data)/(max_data-min_data+1e-6)
X_test =  X_test-0.5

In [51]:
train_dataset = myDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size= 6, shuffle=True)

valid_dataset = myDataset(X_test, y_test)
valid_loader = DataLoader(valid_dataset, batch_size= 6, shuffle=True)

In [52]:
model_ft = Emo2DNet(30)

if torch.cuda.is_available():
    model_ft = model_ft.cuda()

In [53]:
dataset_sizes = {'train':len(train_loader.dataset),'valid':len(valid_loader.dataset)}
dataloaders = {'train':train_loader,'valid':valid_loader}

In [60]:
learning_rate = 0.01
criterion = nn.CrossEntropyLoss()
#optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
optimizer_ft = optim.Adam(model_ft.parameters(), lr=0.001)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=10, gamma=0.1)

In [61]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=24)

Epoch 0/23
----------


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:123: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


train Loss: 0.1593 Acc: 0.6524
valid Loss: 0.4913 Acc: 0.1488

Epoch 1/23
----------
train Loss: 0.1579 Acc: 0.6516
valid Loss: 0.5652 Acc: 0.0970

Epoch 2/23
----------
train Loss: 0.1553 Acc: 0.6588
valid Loss: 0.5787 Acc: 0.1222

Epoch 3/23
----------
train Loss: 0.1565 Acc: 0.6523
valid Loss: 0.5848 Acc: 0.1046

Epoch 4/23
----------
train Loss: 0.1557 Acc: 0.6621
valid Loss: 0.6401 Acc: 0.0894

Epoch 5/23
----------
train Loss: 0.1503 Acc: 0.6757
valid Loss: 0.6619 Acc: 0.1056

Epoch 6/23
----------
train Loss: 0.1495 Acc: 0.6710
valid Loss: 0.6061 Acc: 0.1267

Epoch 7/23
----------
train Loss: 0.1489 Acc: 0.6688
valid Loss: 0.6464 Acc: 0.1025

Epoch 8/23
----------
train Loss: 0.1469 Acc: 0.6755
valid Loss: 0.7511 Acc: 0.0918

Epoch 9/23
----------
train Loss: 0.1326 Acc: 0.7080
valid Loss: 0.6465 Acc: 0.0953

Epoch 10/23
----------
train Loss: 0.1297 Acc: 0.7155
valid Loss: 0.6589 Acc: 0.0932

Epoch 11/23
----------
train Loss: 0.1267 Acc: 0.7195
valid Loss: 0.6327 Acc: 0.1018

